In [100]:
import pandas as pd
import numpy as np
import xgboost as xgb # Usaremos XGBoost por su rapidez y manejo de nulos [cite: 395]
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuración visual
plt.style.use('default')
sns.set_palette("husl")

# 1. Cargar el Dataset Maestro con Clusters
# Ajusta la ruta si es necesario
try:
    df = pd.read_parquet("data_cleaned/master_with_clusters.parquet")
except:
    df = pd.read_parquet("../data_cleaned/master_with_clusters.parquet")

# Asegurar formato fecha
df['Fecha'] = pd.to_datetime(df['Fecha'])

print(f"Datos cargados: {df.shape}")
print(f"Fechas: {df['Fecha'].min()} a {df['Fecha'].max()}")

Datos cargados: (3000, 27)
Fechas: 2024-01-31 00:00:00 a 2024-12-30 00:00:00


In [101]:
# ==========================================
# 2. FEATURE ENGINEERING CORREGIDO
# ==========================================

# A) Agrupación Semanal (Target)
df['year'] = df['Fecha'].dt.year
df['week'] = df['Fecha'].dt.isocalendar().week

# 1. Influencia de Clientes (Pivot)
influence_clientes = df.groupby(['year', 'week', 'Categoría', 'Cluster_Producto', 'Cluster_Cliente'])['Cantidad'].sum().unstack(fill_value=0)
influence_clientes.columns = [f'volumen_clientes_cluster_{c}' for c in influence_clientes.columns]
influence_clientes = influence_clientes.reset_index()

# 2. Efecto Quincena/Fin de Mes
dates_ref = df[['Fecha', 'year', 'week']].drop_duplicates()
dates_ref['es_quincena'] = dates_ref['Fecha'].dt.day.isin([15, 30, 31, 1, 25, 26, 27, 28, 29]).astype(int)
payday_impact = dates_ref.groupby(['year', 'week'])['es_quincena'].sum().reset_index()
payday_impact.columns = ['year', 'week', 'dias_pago_semana']

# 3. Dataset Base Semanal
df_weekly = df.groupby(['year', 'week', 'Categoría', 'Cluster_Producto']).agg({
    'Cantidad': 'sum',
    'Precio_Unitario': 'mean',
    'Stock': 'mean',
    'Fecha': 'max'
}).reset_index()

# 4. Merges (Uniones)
# Unir efecto quincena
df_weekly = df_weekly.merge(payday_impact, on=['year', 'week'], how='left')

# Calcular cambio de precio (Elasticidad)
df_weekly = df_weekly.sort_values(['Categoría', 'Cluster_Producto', 'year', 'week'])
df_weekly['precio_pct_change'] = df_weekly.groupby(['Categoría', 'Cluster_Producto'])['Precio_Unitario'].pct_change().fillna(0)

# Unir influencia de clientes
df_final = df_weekly.merge(influence_clientes, on=['year', 'week', 'Categoría', 'Cluster_Producto'], how='left').fillna(0)

# B) FEATURES AVANZADAS (AQUÍ ES DONDE ESTABA EL ERROR)
def add_advanced_features(df):
    df = df.copy()
    # Asegurar orden
    df = df.sort_values(['Categoría', 'Cluster_Producto', 'year', 'week'])
    
    g = df.groupby(['Categoría', 'Cluster_Producto'])
    
    # 1. Rolling Mean (Tendencia)
    df['rolling_mean_4w'] = g['Cantidad'].transform(lambda x: x.shift(1).rolling(window=4).mean())
    
    # 2. Precio Relativo
    avg_price_8w = g['Precio_Unitario'].transform(lambda x: x.shift(1).rolling(window=8).mean())
    df['precio_relativo'] = df['Precio_Unitario'] / (avg_price_8w + 1e-6)
    
    # 3. Probabilidad de Venta Reciente
    df['prob_venta_reciente'] = g['Cantidad'].transform(lambda x: (x.shift(1) > 0).rolling(window=4).mean())
    
    return df.fillna(0)

# --- CORRECCIÓN AQUÍ: Usamos df_final, no df_model ---
print("Generando features avanzadas...")
df_enriched = add_advanced_features(df_final)

# C) PREPARACIÓN FINAL (Encoding y Cíclicas)
# Variables cíclicas de tiempo
df_enriched['week_sin'] = np.sin(2 * np.pi * df_enriched['week']/52)
df_enriched['week_cos'] = np.cos(2 * np.pi * df_enriched['week']/52)

# One-Hot Encoding de Categoría (Ahora sí, al final)
df_ready = pd.get_dummies(df_enriched, columns=['Categoría'], prefix='Cat')

# D) CREACIÓN DE LAGS
def create_lags(df, target_col, lags):
    df_lagged = df.copy()
    # Creamos un ID temporal para agrupar (Cluster de producto)
    df_lagged['group_id'] = df_lagged['Cluster_Producto'].astype(str)
    df_lagged = df_lagged.sort_values(['year', 'week'])
    
    for lag in lags:
        # Lag del target (Cantidad)
        df_lagged[f'lag_{lag}_{target_col}'] = df_lagged.groupby('group_id')[target_col].shift(lag)
        
        # Lag de Clientes VIP (Cluster 1) - Muy importante
        if 'volumen_clientes_cluster_1' in df_lagged.columns:
             df_lagged[f'lag_{lag}_vip'] = df_lagged.groupby('group_id')['volumen_clientes_cluster_1'].shift(lag)
             
    return df_lagged

# Aplicar Lags
df_model_final = create_lags(df_ready, 'Cantidad', [1, 2, 3, 4])
df_model_advanced = df_model_final.dropna()

print(f"Dataset listo para entrenar: {df_model_advanced.shape}")
print("Columnas clave:", df_model_advanced.columns[-10:].tolist())

Generando features avanzadas...
Dataset listo para entrenar: (563, 34)
Columnas clave: ['Cat_Panadería', 'group_id', 'lag_1_Cantidad', 'lag_1_vip', 'lag_2_Cantidad', 'lag_2_vip', 'lag_3_Cantidad', 'lag_3_vip', 'lag_4_Cantidad', 'lag_4_vip']


In [102]:
# 1. Definimos explícitamente qué NO podemos ver del futuro/presente
columnas_prohibidas = [
    'Cantidad',                     # El target
    'Fecha', 'year', 'week', 'group_id',  # Identificadores
    'volumen_clientes_cluster_0',   # LEAKAGE: Ventas actuales Cluster 0
    'volumen_clientes_cluster_1',   # LEAKAGE: Ventas actuales Cluster 1
    'volumen_clientes_cluster_2',   # LEAKAGE: Ventas actuales Cluster 2
    # Opcional: Si Stock varía mucho por las ventas mismas, mejor quítalo también
    # 'Stock' 
]

# 2. Seleccionamos solo las features válidas (Numéricas - Prohibidas)
features_numericas = df_model_advanced.select_dtypes(include=[np.number]).columns.tolist()
features_finales = [col for col in features_numericas if col not in columnas_prohibidas]

print(f"Entrenando con {len(features_finales)} variables limpias.")
print("Variables incluidas (Muestra):", features_finales[:10])
print("¿Está 'lag_1_vip' incluido? ", 'lag_1_vip' in features_finales) # Debe ser True
print("¿Está 'volumen_clientes_cluster_1' incluido? ", 'volumen_clientes_cluster_1' in features_finales) # Debe ser False

# ==============================================================================
# RE-ENTRENAMIENTO (Copia exacta de tu lógica de split y train)
# ==============================================================================

# Actualizamos la lista global de features para que la función la use
features = features_finales 

# Split Temporal
weeks = df_model_advanced['week'].unique()
split_date = df_model_advanced['Fecha'].max() - pd.Timedelta(weeks=8)

train_adv = df_model_advanced[df_model_advanced['Fecha'] <= split_date]
test_adv = df_model_advanced[df_model_advanced['Fecha'] > split_date]

Entrenando con 18 variables limpias.
Variables incluidas (Muestra): ['Cluster_Producto', 'Precio_Unitario', 'Stock', 'dias_pago_semana', 'precio_pct_change', 'rolling_mean_4w', 'precio_relativo', 'prob_venta_reciente', 'week_sin', 'week_cos']
¿Está 'lag_1_vip' incluido?  True
¿Está 'volumen_clientes_cluster_1' incluido?  False


In [107]:
# ==============================================================================
# ENTRENAMIENTO FINAL CON PARÁMETROS GANADORES (OPTUNA)
# ==============================================================================

def train_final_optimized(df_train, df_test, cluster_id):
    print(f"\nEntrenando Modelo FINAL para Cluster {cluster_id}...")
    
    # Filtro de datos
    train_subset = df_train[df_train['Cluster_Producto'] == cluster_id]
    test_subset = df_test[df_test['Cluster_Producto'] == cluster_id]
    
    X_train = train_subset[features]
    y_train = train_subset[target]
    X_test = test_subset[features]
    y_test = test_subset[target]
    
    # PARÁMETROS EXACTOS ENCONTRADOS
    if cluster_id == 1:
        # Cluster 1: Precisión Lenta (Poisson-like)
        params = {
            'objective': 'reg:tweedie',
            'tweedie_variance_power': 1.11,
            'n_estimators': 782,
            'learning_rate': 0.0055,
            'max_depth': 7,
            'colsample_bytree': 0.73,
            'subsample': 0.98,
            'min_child_weight': 7,
            'n_jobs': -1,
            'eval_metric': "mae",
            'random_state': 42,
            'early_stopping_rounds': 22
        }
    else:
        # Cluster 0: Varianza Alta (Gamma-like)
        params = {
            'objective': 'reg:tweedie',
            'tweedie_variance_power': 1.71,
            'n_estimators': 2533,
            'learning_rate': 0.077,
            'max_depth': 8,
            'colsample_bytree': 0.68,
            'subsample': 0.65,
            'min_child_weight': 1,
            'n_jobs': -1,
            'eval_metric': "mae",
            'random_state': 42,
            'early_stopping_rounds': 34
        }

    model = xgb.XGBRegressor(**params)
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=False
    )
    
    preds = model.predict(X_test)
    preds = np.maximum(preds, 0)
    
    # Guardar resultados
    result_df = test_subset.copy()
    result_df['prediccion_stock'] = preds
    
    wmape = np.sum(np.abs(y_test - preds)) / np.sum(y_test)
    print(f"--> WMAPE Final Cluster {cluster_id}: {wmape:.1%}")
    
    return result_df

# Ejecución Final y Exportación
print("Generando predicciones finales de stock...")
res_0 = train_final_optimized(train_adv, test_adv, cluster_id=0)
res_1 = train_final_optimized(train_adv, test_adv, cluster_id=1)

df_forecast_stock = pd.concat([res_0, res_1])


Generando predicciones finales de stock...

Entrenando Modelo FINAL para Cluster 0...
--> WMAPE Final Cluster 0: 39.9%

Entrenando Modelo FINAL para Cluster 1...
--> WMAPE Final Cluster 1: 31.4%


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# 1. PREPARACIÓN Y REPARACIÓN DE DATOS
# ==============================================================================
results = df_forecast_stock.copy()

# REPARACIÓN DEL ERROR 'Categoría': Reconstruimos la columna desde las dummies
# Identificamos las columnas que empiezan con 'Cat_'
cat_cols = [c for c in results.columns if c.startswith('Cat_')]

# Buscamos cuál tiene el valor 1 y extraemos el nombre
if len(cat_cols) > 0:
    results['Categoría'] = results[cat_cols].idxmax(axis=1).str.replace('Cat_', '')
else:
    # Fallback por si acaso (aunque no debería pasar si corriste el código anterior)
    results['Categoría'] = 'Desconocida'

# Agrupación temporal para la gráfica de línea
weekly_agg = results.groupby('Fecha')[['Cantidad', 'prediccion_stock']].sum().reset_index()

# Cálculo de WMAPE por Categoría para la gráfica de barras
cat_performance = results.groupby('Categoría').apply(
    lambda x: np.sum(np.abs(x['Cantidad'] - x['prediccion_stock'])) / np.sum(x['Cantidad'])
).reset_index(name='WMAPE').sort_values('WMAPE')

# Filtro para Scatter Plot (Solo productos VIP - Cluster 1)
vip_data = results[results['Cluster_Producto'] == 1]

# ==============================================================================
# 2. DASHBOARD INTERACTIVO CON PLOTLY
# ==============================================================================

# Crear estructura del Dashboard (2 filas: 1 arriba, 2 abajo)
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"colspan": 2}, None], [{}, {}]],
    subplot_titles=("Evolución de la Demanda Total (Real vs Pronóstico)", 
                    "Precisión por Categoría (WMAPE)", 
                    "Ajuste en Productos Estrella (Cluster 1)"),
    vertical_spacing=0.15
)

# --- GRÁFICA 1: LÍNEA DE TIEMPO (TENDENCIA) ---
fig.add_trace(
    go.Scatter(x=weekly_agg['Fecha'], y=weekly_agg['Cantidad'], 
               name="Demanda Real", mode='lines+markers', line=dict(color='#2c3e50', width=3)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=weekly_agg['Fecha'], y=weekly_agg['prediccion_stock'], 
               name="Pronóstico Modelo", mode='lines+markers', line=dict(color='#27ae60', width=3, dash='dash')),
    row=1, col=1
)

# --- GRÁFICA 2: BARRAS DE ERROR (WMAPE) ---
# Colores dinámicos según el error (Verde < 30%, Amarillo < 40%, Rojo > 40%)
colors = ['#27ae60' if x < 0.3 else '#f1c40f' if x < 0.4 else '#e74c3c' for x in cat_performance['WMAPE']]

fig.add_trace(
    go.Bar(
        x=cat_performance['Categoría'], 
        y=cat_performance['WMAPE'],
        text=cat_performance['WMAPE'].apply(lambda x: f'{x:.1%}'),
        textposition='auto',
        marker_color=colors,
        name="Error WMAPE"
    ),
    row=2, col=1
)

# --- GRÁFICA 3: SCATTER PLOT (PRECISIÓN VIP) ---
fig.add_trace(
    go.Scatter(
        x=vip_data['Cantidad'], 
        y=vip_data['prediccion_stock'],
        mode='markers',
        marker=dict(color='#3498db', opacity=0.6, size=7),
        name="Productos VIP",
        hovertemplate="Real: %{x}<br>Pred: %{y:.1f}<br>Producto: %{text}",
        text=vip_data['Categoría'] # Opcional: podrías poner ID_Producto si lo recuperamos
    ),
    row=2, col=2
)

# Línea de perfección para el scatter
max_val = max(vip_data['Cantidad'].max(), vip_data['prediccion_stock'].max())
fig.add_trace(
    go.Scatter(x=[0, max_val], y=[0, max_val], mode='lines', 
               line=dict(color='red', dash='dot'), name="Predicción Perfecta"),
    row=2, col=2
)

# Actualizar diseño general
fig.update_layout(
    title_text="<b>Dashboard Ejecutivo de Pronóstico de Stock</b>",
    title_x=0.5,
    height=900,
    width=1200,
    showlegend=True,
    template="plotly_white"
)

# Ejes y formatos
fig.update_yaxes(title_text="Unidades", row=1, col=1)
fig.update_yaxes(title_text="Error (WMAPE)", tickformat=".0%", row=2, col=1)
fig.update_xaxes(title_text="Demanda Real", row=2, col=2)
fig.update_yaxes(title_text="Pronóstico", row=2, col=2)

fig.show()

KeyError: 'Categoría'

In [109]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# 1. Preparar Datos (Igual que antes)
# Agrupación temporal
weekly_agg = results.groupby('Fecha')[['Cantidad', 'prediccion_stock']].sum().reset_index()

# Métricas por Categoría
cat_performance = results.groupby('Categoría').apply(
    lambda x: np.sum(np.abs(x['Cantidad'] - x['prediccion_stock'])) / np.sum(x['Cantidad'])
).reset_index(name='WMAPE').sort_values('WMAPE')

# Datos por Cluster
vip_data = results[results['Cluster_Producto'] == 1]
slow_data = results[results['Cluster_Producto'] == 0]

# 2. Configurar Dashboard 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "1. Evolución Global de Demanda (Real vs Pronóstico)", 
        "2. Precisión por Categoría (Menor es Mejor)", 
        "3. Ajuste: Productos Lentos (Cluster 0)",
        "4. Ajuste: Productos Estrella (Cluster 1)"
    ),
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# --- Fila 1, Col 1: LÍNEA DE TIEMPO ---
fig.add_trace(
    go.Scatter(x=weekly_agg['Fecha'], y=weekly_agg['Cantidad'], 
               name="Real", mode='lines+markers', line=dict(color='#2c3e50', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=weekly_agg['Fecha'], y=weekly_agg['prediccion_stock'], 
               name="Pronóstico", mode='lines+markers', line=dict(color='#27ae60', width=2, dash='dash')),
    row=1, col=1
)

# --- Fila 1, Col 2: BARRAS WMAPE ---
colors = ['#27ae60' if x < 0.3 else '#f1c40f' if x < 0.4 else '#e74c3c' for x in cat_performance['WMAPE']]
fig.add_trace(
    go.Bar(
        x=cat_performance['Categoría'], y=cat_performance['WMAPE'],
        text=cat_performance['WMAPE'].apply(lambda x: f'{x:.1%}'),
        textposition='auto', marker_color=colors, showlegend=False
    ),
    row=1, col=2
)

# --- Fila 2, Col 1: SCATTER CLUSTER 0 (Lentos) ---
max_val_0 = max(slow_data['Cantidad'].max(), slow_data['prediccion_stock'].max())
fig.add_trace(
    go.Scatter(x=slow_data['Cantidad'], y=slow_data['prediccion_stock'], mode='markers',
               marker=dict(color='#95a5a6', opacity=0.5, size=6), name="Prod. Lentos"),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=[0, max_val_0], y=[0, max_val_0], mode='lines', 
               line=dict(color='red', dash='dot'), showlegend=False),
    row=2, col=1
)

# --- Fila 2, Col 2: SCATTER CLUSTER 1 (Estrella) ---
max_val_1 = max(vip_data['Cantidad'].max(), vip_data['prediccion_stock'].max())
fig.add_trace(
    go.Scatter(x=vip_data['Cantidad'], y=vip_data['prediccion_stock'], mode='markers',
               marker=dict(color='#3498db', opacity=0.6, size=7), name="Prod. Estrella"),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=[0, max_val_1], y=[0, max_val_1], mode='lines', 
               line=dict(color='red', dash='dot'), showlegend=False),
    row=2, col=2
)

# Layout Final
fig.update_layout(height=800, width=1200, title_text="<b>Dashboard de Resultados del Modelo de Stock</b>", template="plotly_white")
fig.update_yaxes(title_text="Unidades", row=1, col=1)
fig.update_yaxes(title_text="Error WMAPE", tickformat=".0%", row=1, col=2)
fig.update_xaxes(title_text="Demanda Real", row=2, col=1)
fig.update_yaxes(title_text="Pronóstico", row=2, col=1)
fig.update_xaxes(title_text="Demanda Real", row=2, col=2)
fig.update_yaxes(title_text="Pronóstico", row=2, col=2)

fig.show()

In [106]:
# import optuna
# import xgboost as xgb
# from sklearn.metrics import mean_absolute_error

# def objective(trial):
#     # Definir el espacio de búsqueda de parámetros
#     params = {
#         'objective': 'reg:tweedie',
#         'tweedie_variance_power': trial.suggest_float('tweedie_variance_power', 1.1, 1.9),
#         'n_estimators': trial.suggest_int('n_estimators', 500, 3000),
#         'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         'n_jobs': -1,
#         'eval_metric': "mae",
#         'random_state': 42,
#         'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 20, 100)
#     }
#     
#     # Entrenar con estos parámetros (usando solo Cluster 1 para la prueba)
#     cluster_id = 1
#     train_subset = train_adv[train_adv['Cluster_Producto'] == cluster_id]
#     test_subset = test_adv[test_adv['Cluster_Producto'] == cluster_id]
#     
#     model = xgb.XGBRegressor(**params)
#     
#     model.fit(
#         train_subset[features], train_subset[target],
#         eval_set=[(test_subset[features], test_subset[target])],
#         verbose=False,
#  
#     )
#     
#     preds = model.predict(test_subset[features])
#     preds = np.maximum(preds, 0)
#     
#     # Optimizamos para WMAPE
#     wmape = np.sum(np.abs(test_subset[target] - preds)) / np.sum(test_subset[target])
#     return wmape

# print("Iniciando búsqueda de hiperparámetros para Cluster 1...")
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100) # 20 intentos (puedes subirlo a 50 si tienes tiempo)

# print("Mejores parámetros encontrados:")
# print(study.best_params)
# print(f"Mejor WMAPE posible: {study.best_value:.1%}")